In [ ]:
# Importing the librariies and function initialization #
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
import cv2
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import spatial
import random
import sklearn.metrics as metrics
from skimage.metrics import structural_similarity as ssim
import hashlib
pca = PCA(n_components=15) # PCA using first 15 principal components
import hashlib
resnet_vgg = InceptionResnetV1(pretrained='vggface2').eval()
resnet_casia = InceptionResnetV1(pretrained='casia-webface').eval()
resnet_v3 = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).eval()
alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True).eval()
resnext = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True).eval()
mtcnn = MTCNN()

/opt/homebrew/Caskroom/miniforge/base/envs/tf_clone/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /Users/nemo/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/nemo/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/nemo/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
# Create an embedding using inception resnet and creation of features and targets #

def embedding_creation(newimg,prtrn):

    width = 416
    height = 416
    dim = (width, height)

    img = cv2.imread(newimg)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img_cropped = mtcnn(resized)

    if(prtrn == 1):
        img_embedding = resnet_vgg(img_cropped.unsqueeze(0))
    if(prtrn == 2):
        img_embedding = resnet_casia(img_cropped.unsqueeze(0))
    if(prtrn == 3):
        img_embedding = resnet_v3(img_cropped.unsqueeze(0))
    if(prtrn == 4):
        img_embedding = alexnet(img_cropped.unsqueeze(0))
    if(prtrn == 5):
        img_embedding = resnext(img_cropped.unsqueeze(0))


    img_array = img_embedding.detach().numpy()

    return(img_array)

In [ ]:
def vector_creation(foldername,prtrn):

    datafolder = (os.getcwd() + '/' + str(foldername) + '/')
    mydirs = (os.listdir(datafolder))
    all_data = []
    Y_all = []

    if 'random' in foldername:
        all_random = []
        Y_random = []

        for file in mydirs:
            mypath = os.path.join(datafolder + file)

            try:
                all_random.append(embedding_creation(mypath,prtrn))
                Y_random.append(0)
            except:
                print(file)
    #return(np.shape(all_random))

        all_random = np.array(all_random)
        X_random = all_random.reshape(all_random.shape[0], (all_random.shape[1]*all_random.shape[2]))
        X_random = np.around(X_random,4)

        Y_random = np.array(Y_random)

        return (X_random,Y_random)



    else:

        i = 1

        for mydir in mydirs:

            for file in os.listdir(datafolder + '/' + mydir):
                mypath = os.path.join(datafolder + '/' + mydir + '/'+ file)
                all_data.append(embedding_creation(mypath,prtrn))

                Y_all.append(i)

            i+=1

        all_data = np.array(all_data)
        X_data = all_data.reshape(all_data.shape[0], (all_data.shape[1]*all_data.shape[2]))
        X_data = np.around(X_data,4)

        Y_data = np.array(Y_all)

        return (X_data,Y_data)

In [ ]:
A_data = vector_creation('att_master',1)[0]
B_data = vector_creation('att_master',2)[0]
C_data = vector_creation('att_master',3)[0]
D_data = vector_creation('att_master',4)[0]
E_data = vector_creation('att_master',5)[0]
Y_data = vector_creation('att_master',1)[1]

A_random = vector_creation('2k_random_images',1)[0]
B_random = vector_creation('2k_random_images',2)[0]
C_random = vector_creation('2k_random_images',3)[0]
D_random = vector_creation('2k_random_images',4)[0]
E_random = vector_creation('2k_random_images',5)[0]


In [ ]:
def cosine_similarity_comparison(i,j,dataset,vector):

        # This is for comparison with template class #
    thresh = 0
    if(vector == 'A'):
        sample1 = A_data[i]
        if(dataset ==0):
            sample2 = A_random[j]

        else:
            sample2 = A_data[j]
    if(vector == 'B'):
        sample1 = B_data[i]
        if(dataset ==0):
            sample2 = B_random[j]
        else:
            sample2 = B_data[j]

    if(vector == 'C'):
        sample1 = C_data[i]
        if(dataset ==0):
            sample2 = C_random[j]
        else:
            sample2 = C_data[j]


    if(vector == 'D'):
        sample1 = D_data[i]
        if(dataset ==0):
            sample2 = D_random[j]

        else:
            sample2 = D_data[j]

    if(vector == 'E'):
        sample1 = E_data[i]
        if(dataset ==0):
            sample2 = E_random[j]

        else:
            sample2 = E_data[j]

    result = 1 - spatial.distance.cosine(sample1, sample2)
    return round(result,4)

In [ ]:
### Distribution generation function for class 1 , class 2 and random images ###

template_class_dist_vector_A = []
random_class_dist_vector_A = []

template_class_dist_vector_B = []
random_class_dist_vector_B = []

template_class_dist_vector_C = []
random_class_dist_vector_C = []

template_class_dist_vector_D = []
random_class_dist_vector_D = []

template_class_dist_vector_E = []
random_class_dist_vector_E = []

# (i,j,dataset,vector) #
j = 0

for j in range(0,400,10):
    i = 0
    for i in range(j,j+10):

        template_class_dist_vector_A.append(cosine_similarity_comparison(j,i,1,'A'))
        random_class_dist_vector_A.append(cosine_similarity_comparison(j,i,0,'A'))

        template_class_dist_vector_B.append(cosine_similarity_comparison(j,i,1,'B'))
        random_class_dist_vector_B.append(cosine_similarity_comparison(j,i,0,'B'))

        template_class_dist_vector_C.append(cosine_similarity_comparison(j,i,1,'C'))
        random_class_dist_vector_C.append(cosine_similarity_comparison(j,i,0,'C'))

        template_class_dist_vector_D.append(cosine_similarity_comparison(j,i,1,'D'))
        random_class_dist_vector_D.append(cosine_similarity_comparison(j,i,0,'D'))


        template_class_dist_vector_E.append(cosine_similarity_comparison(j,i,1,'E'))
        random_class_dist_vector_E.append(cosine_similarity_comparison(j,i,0,'E'))

In [ ]:
len(random_class_dist_vector_A)

In [ ]:
### Histogram plotting ###

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

axs[0].hist(template_class_dist_vector_A, bins=10)
axs[1].hist(random_class_dist_vector_A, bins=10)

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

axs[0].hist(template_class_dist_vector_B, bins=10)
axs[1].hist(random_class_dist_vector_B, bins=10)

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

axs[0].hist(template_class_dist_vector_C, bins=10)
axs[1].hist(random_class_dist_vector_C, bins=10)

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

axs[0].hist(template_class_dist_vector_D, bins=10)
axs[1].hist(random_class_dist_vector_D, bins=10)

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

axs[0].hist(template_class_dist_vector_E, bins=10)
axs[1].hist(random_class_dist_vector_E, bins=10)


In [ ]:
## plotting ROC for various condition ##

def roc_creation(distro1,distro2):

    genuine_label = list(np.ones(400, dtype = int))
    imposter_label = list(np.zeros(400, dtype = int))
    y_test = np.concatenate([np.array(genuine_label), np.array(imposter_label)])

    # return y_test

    genuine_label = list(distro1)
    imposter_label = list(distro2)
    y_prob = np.concatenate([np.array(genuine_label), np.array(imposter_label)])



    fpr, tpr, threshold = metrics.roc_curve(y_test, y_prob)
    roc_auc = metrics.auc(fpr, tpr)



    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
roc_creation(template_class_dist_vector_E,random_class_dist_vector_A)
roc_creation(template_class_dist_vector_E,random_class_dist_vector_B)
roc_creation(template_class_dist_vector_E,random_class_dist_vector_C)
roc_creation(template_class_dist_vector_E,random_class_dist_vector_D)
roc_creation(template_class_dist_vector_E,random_class_dist_vector_E)

In [ ]:
# new hash generation function #
def newhash(mystring):

    encoded=mystring.encode()
    result = hashlib.sha256(encoded)

    #print("Block Size : ", end ="")
    #print(result.block_size)

    return(result.hexdigest())

In [ ]:
# generic hash function testing #

fixed_hash = 'a36b1f2c3f84522dd1005145646617d7054c0851e97c72a039c0bdfac9fa07f3'

current_hash = newhash(str([1,2,3]))


if(current_hash == fixed_hash):
    print('Hash function is working properly')
else:
    print("There is some issue with hash function")

In [ ]:
# template image hash generation #

i = 0
A0 = []
B0 = []
C0 = []
D0 = []
E0 = []

for i in range(0,400,10):
    A0.append(A_data[i])
    B0.append(B_data[i])
    C0.append(C_data[i])
    D0.append(D_data[i])
    E0.append(E_data[i])

In [ ]:
m0 = newhash(str([A0[0],B0[1],C0[0],D0[0],E0[0]]))

In [ ]:
# advanced hash function testing #

if (m0 == 'bbd58fca367aec3ac6f0d9d8d52297cdc8a1ae532037ac1480c2b16120394ac2'):
    print("hash generation of vector points is working fine")
else:
    print("please check hash generation of vector points")

In [ ]:
### key generation ###

key_master = []

for i in range(400):
    key_master.append(newhash(str([A_data[i],B_data[i],C_data[i],D_data[i],E_data[i]])))

In [ ]:
### vault preparation ###

vault_A_cosines = []
vault_B_cosines = []
vault_C_cosines = []
vault_D_cosines = []
vault_E_cosines = []


vault_A_2k1 = []
vault_B_2k1 = []
vault_C_2k1 = []
vault_D_2k1 = []
vault_E_2k1 = []



matched = 0
unmatched = 0

j=0

for j in range(400):

    vault_A_cosines = []
    vault_B_cosines = []
    vault_C_cosines = []
    vault_D_cosines = []
    vault_E_cosines = []

    vault_A_2k1 = []
    vault_B_2k1 = []
    vault_C_2k1 = []
    vault_D_2k1 = []
    vault_E_2k1 = []


    vault_A_cosines.append(1 - spatial.distance.cosine(A_data[j], A_data[j]))
    vault_B_cosines.append(1 - spatial.distance.cosine(B_data[j], B_data[j]))
    vault_C_cosines.append(1 - spatial.distance.cosine(C_data[j], C_data[j]))
    vault_D_cosines.append(1 - spatial.distance.cosine(D_data[j], D_data[j]))
    vault_E_cosines.append(1 - spatial.distance.cosine(E_data[j], E_data[j]))

    vault_A_2k1.append(A_data[j])
    vault_B_2k1.append(B_data[j])
    vault_C_2k1.append(C_data[j])
    vault_D_2k1.append(D_data[j])
    vault_E_2k1.append(E_data[j])

    i = 0
    for i in range(2000):

        vault_A_cosines.append(1 - spatial.distance.cosine(A_data[j], A_random[i]))
        vault_B_cosines.append(1 - spatial.distance.cosine(B_data[j], B_random[i]))
        vault_B_cosines.append(1 - spatial.distance.cosine(C_data[j], C_random[i]))
        vault_D_cosines.append(1 - spatial.distance.cosine(D_data[j], D_random[i]))
        vault_E_cosines.append(1 - spatial.distance.cosine(E_data[j], E_random[i]))


        vault_A_2k1.append(A_random[i])
        vault_B_2k1.append(B_random[i])
        vault_C_2k1.append(C_random[i])
        vault_D_2k1.append(D_random[i])
        vault_E_2k1.append(E_random[i])

    index_a = vault_A_cosines.index(max(vault_A_cosines))
    index_b = vault_B_cosines.index(max(vault_B_cosines))
    index_c = vault_C_cosines.index(max(vault_C_cosines))
    index_d = vault_D_cosines.index(max(vault_D_cosines))
    index_e = vault_E_cosines.index(max(vault_E_cosines))

    m_compare = newhash(str([vault_A_2k1[index_a],vault_B_2k1[index_b],vault_C_2k1[index_c],vault_D_2k1[index_d],vault_E_2k1[index_e]]))

    if(key_master[j] == m_compare):
        matched += 1
    else:
        unmatched += 1

In [ ]:
matched